In [ ]:
! pip install --upgrade xarray zarr gcsfs cftime pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np


In [ ]:
# CMIP6 Zarr paths for ACCESS-CM2 model
HIST_ZARR_PATH = 'gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/Amon/clt/gn/v20191108/'
FUTURE_ZARR_PATH = 'gs://cmip6/CMIP6/ScenarioMIP/CSIRO-ARCCSS/ACCESS-CM2/ssp585/r1i1p1f1/Amon/clt/gn/v20191108/'

# Study region: Southern California coastal box
LAT_MIN, LAT_MAX = 30, 35
LON_MIN, LON_MAX = -125, -115

Updated HIST_ZARR_PATH: gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Amon/clt/gr1/v20190726/
Updated FUTURE_ZARR_PATH: gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp370/r1i1p1f1/Amon/clt/gr1/v20180701/


In [ ]:
# Load CMIP6 data from Google Cloud
ds_hist = xr.open_zarr(HIST_ZARR_PATH, consolidated=False, storage_options={'anon': True})
ds_fut = xr.open_zarr(FUTURE_ZARR_PATH, consolidated=False, storage_options={'anon': True})

# Convert longitude from 0-360 to -180-180
def convert_longitude(ds):
    ds['lon'] = xr.where(ds['lon'] > 180, ds['lon'] - 360, ds['lon'])
    return ds.sel(lon=sorted(ds['lon'].values))

ds_hist = convert_longitude(ds_hist)
ds_fut = convert_longitude(ds_fut)

# Subset to Southern California region
ds_hist_subset = ds_hist.sel(lat=slice(LAT_MIN, LAT_MAX), lon=slice(LON_MIN, LON_MAX))
ds_fut_subset = ds_fut.sel(lat=slice(LAT_MIN, LAT_MAX), lon=slice(LON_MIN, LON_MAX))

print(f"Historical: {len(ds_hist_subset.time)} months, {ds_hist_subset.sizes['lat']}x{ds_fut_subset.sizes['lon']} grid")
print(f"Future: {len(ds_fut_subset.time)} months, {ds_fut_subset.sizes['lat']}x{ds_fut_subset.sizes['lon']} grid")

Loading historical data from Zarr: gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Amon/clt/gr1/v20190726/
Loading future data from Zarr: gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp370/r1i1p1f1/Amon/clt/gr1/v20180701/
Historical Dataset loaded:
<xarray.Dataset> Size: 411MB
Dimensions:    (time: 1980, lat: 180, lon: 288, bnds: 2)
Coordinates:
  * time       (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 2kB 0.625 1.875 3.125 4.375 ... 356.9 358.1 359.4
  * bnds       (bnds) float64 16B 1.0 2.0
    lat_bnds   (lat, bnds) float64 3kB dask.array<chunksize=(180, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 5kB dask.array<chunksize=(288, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
Data variables:
    clt        (time, lat, lon) float32 411MB dask.array<chunksize=(346, 1

In [ ]:
# Extract monthly data and compute spatial average
clt_hist_monthly = ds_hist_subset['clt'].mean(dim=['lat', 'lon'])
clt_fut_monthly = ds_fut_subset['clt'].mean(dim=['lat', 'lon'])

# Convert to DataFrames
def process_to_df(clt_data, scenario_name):
    df = clt_data.to_dataframe().reset_index()
    df['year'] = df['time'].apply(lambda x: x.year if hasattr(x, 'year') else pd.to_datetime(x).year)
    df['month'] = df['time'].apply(lambda x: x.month if hasattr(x, 'month') else pd.to_datetime(x).month)
    df['scenario'] = scenario_name
    return df[['year', 'month', 'clt', 'scenario']]

hist_monthly_df = process_to_df(clt_hist_monthly, 'historical')
fut_monthly_df = process_to_df(clt_fut_monthly, 'future')
monthly_df = pd.concat([hist_monthly_df, fut_monthly_df], ignore_index=True)
monthly_df = monthly_df.sort_values(['scenario', 'year', 'month']).reset_index(drop=True)

monthly_df.to_csv('data/fog_socal_monthly.csv', index=False)
print(f"Monthly data: {len(monthly_df)} records saved to data/fog_socal_monthly.csv")

Longitude conversion applied to ds_hist and ds_fut.


In [ ]:
# Compute annual means
annual_df = monthly_df.groupby(['scenario', 'year'])['clt'].mean().reset_index()
annual_df.to_csv('data/fog_socal_timeseries.csv', index=False)
print(f"Annual data: {len(annual_df)} records saved to data/fog_socal_timeseries.csv")

ds_hist new longitude range:
Min longitude: -179.375
Max longitude: 179.375

ds_fut new longitude range:
Min longitude: -179.375
Max longitude: 179.375


In [ ]:
# Compute seasonal means (DJF, MAM, JJA, SON)
# Note: DJF includes Dec from previous year, Jan-Feb from current year
def assign_season_year(year, month):
    if month == 12:
        return year + 1  # December belongs to next year's winter
    return year

def get_season(month):
    if month in [12, 1, 2]:
        return 'DJF'
    elif month in [3, 4, 5]:
        return 'MAM'
    elif month in [6, 7, 8]:
        return 'JJA'
    else:
        return 'SON'

monthly_df['season'] = monthly_df['month'].apply(get_season)
monthly_df['season_year'] = monthly_df.apply(lambda x: assign_season_year(x['year'], x['month']), axis=1)
seasonal_df = monthly_df.groupby(['scenario', 'season_year', 'season'])['clt'].mean().reset_index()
seasonal_df.rename(columns={'season_year': 'year'}, inplace=True)
# Save without year for compatibility (aggregated across all years)
seasonal_agg = seasonal_df.groupby(['scenario', 'season'])['clt'].mean().reset_index()
seasonal_agg.to_csv('data/fog_socal_seasonal.csv', index=False)
print(f"Seasonal data: {len(seasonal_agg)} records saved to data/fog_socal_seasonal.csv")

Subset Region: Latitude (30 to 35 N), Longitude (-125 to -115 E)

Historical Dataset Subset loaded:
<xarray.Dataset> Size: 365kB
Dimensions:    (time: 1980, lat: 5, lon: 8, bnds: 2)
Coordinates:
  * time       (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lat        (lat) float64 40B 30.5 31.5 32.5 33.5 34.5
  * lon        (lon) float64 64B -124.4 -123.1 -121.9 ... -118.1 -116.9 -115.6
  * bnds       (bnds) float64 16B 1.0 2.0
    lat_bnds   (lat, bnds) float64 80B dask.array<chunksize=(5, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 128B dask.array<chunksize=(8, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
Data variables:
    clt        (time, lat, lon) float32 317kB dask.array<chunksize=(346, 5, 8), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0

In [ ]:
# Verify output files
print("\nAll CSV files generated:")
print(f"✓ fog_socal_timeseries.csv: {len(annual_df)} records")
print(f"✓ fog_socal_monthly.csv: {len(monthly_df)} records")
print(f"✓ fog_socal_seasonal.csv: {len(seasonal_agg)} records")
print(f"\nData ranges:")
print(f"  Years: {annual_df['year'].min():.0f}-{annual_df['year'].max():.0f}")
print(f"  Cloud fraction: {monthly_df['clt'].min():.1f}% - {monthly_df['clt'].max():.1f}%")

<xarray.Dataset> Size: 190kB
Dimensions:    (time: 1032, lat: 5, lon: 8, bnds: 2)
Coordinates:
  * time       (time) object 8kB 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
  * lat        (lat) float64 40B 30.5 31.5 32.5 33.5 34.5
  * lon        (lon) float64 64B -124.4 -123.1 -121.9 ... -118.1 -116.9 -115.6
  * bnds       (bnds) float64 16B 1.0 2.0
    lat_bnds   (lat, bnds) float64 80B dask.array<chunksize=(5, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object 17kB dask.array<chunksize=(1032, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 128B dask.array<chunksize=(8, 2), meta=np.ndarray>
Data variables:
    clt        (time, lat, lon) float32 165kB dask.array<chunksize=(516, 5, 8), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60225.0
    branch_time_in_parent:  60225.0
    comment:                <null ref>
    ...                     ...
    table_id:               Amon
    title:                  NOAA GFDL GFDL-ESM4 model output prepared for CMI...
    tracking_id:            hdl:21.14100/0c2a80bc-c24f-4c6b-9703-c8f177a5056f
    variable_id:            clt
    variant_info:           N/A
    variant_label:          r1i1p1f1

Historical data: 1980 monthly records
Future data: 1032 monthly records

Monthly data saved to data/fog_socal_monthly.csv
Total shape: (3012, 4)

First few rows:
   year  month        clt scenario
0  2015      1  19.569281   future
1  2015      2  51.559376   future
2  2015      3  46.588493   future
3  2015      4  43.841084   future
4  2015      5  46.465771   future
5  2015      6  38.493099   future
6  2015      7  29.333038   future
7  2015      8  12.781402   future
8  2015      9   9.750980   future
9  2015     10  15.562207   future

Last few rows:
      year  month        clt    scenario
3002  2014      3  32.060604  historical
3003  2014      4  50.829876  historical
3004  2014      5  38.637577  historical
3005  2014      6  43.358040  historical
3006  2014      7  14.829923  historical
3007  2014      8  35.490738  historical
3008  2014      9  25.325159  historical
3009  2014     10  28.275543  historical
3010  2014     11  37.958797  historical
3011  2014     12  52.53422